# Testing Notebook 02:

In [1]:
from typing import List
import pandas as pd 
import numpy as np

from statdepth.depth import BandDepth

NameError: name 'List' is not defined

In [ ]:
!where python

Generate a toy DataFrame for testing

In [4]:
df = pd.DataFrame(np.random.randint(0,5,size=(5, 5)), columns=list('ABCDE'))
df

,A,B,C,D,E
0,3,1,0,1,2
1,2,2,0,2,3
2,0,0,4,0,4
3,1,2,4,2,4
4,0,2,3,0,3


In [6]:
# from statdepth.depth.depth import BandDepth

# bd = BandDepth([df], relax=True)

# bd.ordered()

In [7]:
class Test(pd.Series):
    
    def __init__(self, depths):
        super().__init__(data=depths)
        self.depths = depths
        
    def ordered(self):
        return self.depths.sort_values()

t = Test(pd.Series([9, 5, 6, 2, 1]))

In [8]:
s = pd.Series([1,2,3,4,4])
len(s), len(t)

(5, 5)

In [9]:
class _BandDepthSeries(pd.Series):

    def __init__(self, depths):
        super().__init__(data=depths)

        self._depths = depths
        self._ordered_depths_ = None
        self._deepest_curve_ = None

    def ordered(self, ascending=False):
        if self._ordered_depths_ is None:
            self._ordered_depths_ =  self._depths.sort_values(ascending=ascending)
        return self._ordered_depths_

    def deepest(self):
        if self._ordered_depths_ is not None or (self._ordered_depths_ is None and self._deepest_curve_ is None):
            sorted_depths = self._depths.sort_values(ascending=False)
            self._deepest_curve_ = pd.Series(index=[sorted_depths.index[0]], data=[sorted_depths[0]])
        return self._deepest_curve_
    
class _BandDepthDataFrame(pd.DataFrame):
    def __init__(self):
        pass
    
    
t = _BandDepthSeries(pd.Series([5,1,2,3,6]))

In [12]:
def BandDepth(data: List[pd.DataFrame], K=None, J=2, containment='r2', relax=False, deep_check=False):
    if K is not None:
        data = _samplebanddepth(data=data, K=K, J=J, containment=containment, relax=relax, deep_check=deep_check)
    else:
        data = _banddepth(data=data, J=J, containment=containment, relax=relax, deep_check=deep_check)

    if isinstance(data, pd.DataFrame):
        return _BandDepthDataFrame(data)
    else:
        return _BandDepthSeries(data)


In [13]:
d = BandDepth([df])

In [15]:
type(d)

__main__._BandDepthSeries